In [41]:
from binance import ThreadedWebsocketManager
from binance.client import Client
import pandas as pd
import numpy as np
import datetime

In [9]:
api_key = "DAX5KWi4IudakbEVvmQsjgqRYkSVycWQralfOuIWr7xSyQ4chqn192dLS4QGkG1q"  # TESTING testnet.binance.vision
api_secret = "i5YaSzFjlZTkrqsdMzEGUoukzkKehozhtNi0E5XPVP2UnbzyDuFB8RJuN1JMSTKZ"  # TESTING testnet.binance.vision

In [16]:
client = Client(api_key=api_key, api_secret=api_secret, testnet=True)

In [88]:
client.get_account()

{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'canTrade': True,
 'canWithdraw': False,
 'canDeposit': False,
 'brokered': False,
 'updateTime': 1666462260922,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BNB',
   'free': '1000.00000000',
   'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '1.00000000', 'locked': '0.00000000'},
  {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '9996.04744177', 'locked': '0.00000000'},
  {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}],
 'permissions': ['SPOT']}

...

In [82]:
class LongOnlyTrader():
    
    def __init__(self, symbol, interval, units=0.1, position=0):
        self.symbol = symbol
        self.interval = interval
        self.units = units
        self.position = position
        self.available_intervals = ["1m", "3m", "5m", "15m", "30m", "1h", "2h", "4h", "6h", "8h", "12h", "1d", "3d", "1w", "1M"]

        # Strategy attributs
#         self.return_threshold = return_threshold
#         self.volume_threshold = volume_threshold
        
    def start_trading(self, past_days=2):
        self.twm = ThreadedWebsocketManager()
        self.twm.start()
        
        if self.interval in self.available_intervals:
            self.get_recent_prices(past_days)
            self.twm.start_kline_socket(callback=self.stream_candles,
                                        symbol=self.symbol, interval=self.interval)
        # "else" to be added later in the course 
    
    def get_recent_prices(self, past_days):
        now = datetime.datetime.utcnow()
        past = str(now - datetime.timedelta(days=past_days))
        
        bars = client.get_historical_klines(self.symbol, self.interval, past)
        
        df = pd.DataFrame(bars)
        df["Date"] = pd.to_datetime(df.iloc[:,0], unit = "ms")
        df.columns = ["Open Time", "Open", "High", "Low", "Close", "Volume",
                      "Clos Time", "Quote Asset Volume", "Number of Trades",
                      "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore", "Date"]
        df = df[["Date", "Open", "High", "Low", "Close", "Volume"]].copy()
        df.set_index("Date", inplace=True)
        for column in df.columns:
            df[column] = pd.to_numeric(df[column], errors = "coerce")
        df["Complete"] = [True for row in range(len(df)-1)] + [False]
        
        self.data = df

    def stream_candles(self, msg):
        
        # extract the required items from msg
        event_time = pd.to_datetime(msg["E"], unit = "ms")
        start_time = pd.to_datetime(msg["k"]["t"], unit = "ms")
        first   = float(msg["k"]["o"])
        high    = float(msg["k"]["h"])
        low     = float(msg["k"]["l"])
        close   = float(msg["k"]["c"])
        volume  = float(msg["k"]["v"])
        complete=       msg["k"]["x"]
    
        # print out
        print(".", end="", flush=True)
    
        # feed df (add new bar / update latest bar)
        self.data.loc[start_time] = [first, high, low, close, volume, complete]
        
        if complete:
            self.define_strategy()
            self.execute_order()

    def define_strategy(self):
        df = self.data.copy()
        df = df[["Open", "Close"]]
        # Define strategy  --  If last candle is green, buy. If red, sell
        cond = df.Open > df.Close  # Red candle
        
        df["Position"] = 1
        df.loc[cond, "Position"] = 0
        
        self.prepare_data = df.copy()
        
    def execute_order(self):
        if self.prepare_data["Position"].iloc[-1] == 1:  # Buy
            if self.position == 0:
                order = client.create_order(symbol=self.symbol, side="SELL", type="MARKET", quantity=self.units)
#                 order = client.create_order(symbol=self.symbol, side="BUY", type="MARKET", quantity=self.units)
                self.position = 1
                print("BUYING BITCOIN")

        if self.prepare_data["Position"].iloc[-1] == 0: # Sell:
            if self.position == 1:
#                 order = client.create_order(symbol=self.symbol, side="SELL", type="MARKET", quantity=self.units)
                order = client.create_order(symbol=self.symbol, side="BUY", type="MARKET", quantity=self.units)
                self.position = 0
                print("SELLING BITCOIN")

In [83]:
trader = LongOnlyTrader("BTCUSDT", "1m")

In [84]:
trader.start_trading()

...........................................BUYING BITCOIN
.............................SELLING BITCOIN
................................

In [71]:
trader.data

,Open,High,Low,Close,Volume,Complete
Date,,,,,,
2022-10-20 17:18:00,19137.51,19141.68,19132.23,19136.79,6.760908,True
2022-10-20 17:19:00,19137.56,19143.64,19132.65,19137.49,5.348866,True
2022-10-20 17:20:00,19137.92,19139.74,19132.91,19134.97,5.372226,True
2022-10-20 17:21:00,19134.97,19149.69,19132.91,19149.69,7.448774,True
2022-10-20 17:22:00,19146.90,19354.82,10000.00,19161.00,16.813073,True
...,...,...,...,...,...,...
2022-10-22 17:13:00,19196.63,19196.70,19194.82,19194.82,2.163762,True
2022-10-22 17:14:00,19194.82,19195.61,19192.22,19192.50,2.791750,True
2022-10-22 17:15:00,19192.50,19195.82,19192.47,19193.73,3.988553,True


......

In [89]:
trader.twm.stop()

.

CANCEL read_loop


In [77]:
trader.prepare_data

,Open,Close,Position
Date,,,
2022-10-20 17:18:00,19137.51,19136.79,0
2022-10-20 17:19:00,19137.56,19137.49,0
2022-10-20 17:20:00,19137.92,19134.97,0
2022-10-20 17:21:00,19134.97,19149.69,1
2022-10-20 17:22:00,19146.90,19161.00,1
...,...,...,...
2022-10-22 17:15:00,19192.50,19193.73,1
2022-10-22 17:16:00,19193.43,19204.90,1
2022-10-22 17:17:00,19206.19,19203.56,0


.........................BUYING BITCOIN
..............

In [35]:
trader = LongOnlyTrader("BTCUSDT", "1m")

In [37]:
trader.start_trading()

Time: 2022-10-22 15:37:45.343000 | Price: 19219.72
Time: 2022-10-22 15:37:47.383000 | Price: 19220.99
Time: 2022-10-22 15:37:49.439000 | Price: 19220.11
Time: 2022-10-22 15:37:51.491000 | Price: 19220.69
Time: 2022-10-22 15:37:53.498000 | Price: 19219.82
Time: 2022-10-22 15:37:55.687000 | Price: 19219.74
Time: 2022-10-22 15:37:57.748000 | Price: 19219.06
Time: 2022-10-22 15:37:59.821000 | Price: 19219.38
Time: 2022-10-22 15:38:00.003000 | Price: 19219.56
Time: 2022-10-22 15:38:02.158000 | Price: 19219.51
Time: 2022-10-22 15:38:04.223000 | Price: 19219.2
Time: 2022-10-22 15:38:06.255000 | Price: 19219.14
Time: 2022-10-22 15:38:08.296000 | Price: 19219.11
Time: 2022-10-22 15:38:10.334000 | Price: 19218.81
Time: 2022-10-22 15:38:12.617000 | Price: 19219.31
Time: 2022-10-22 15:38:14.680000 | Price: 19218.79
Time: 2022-10-22 15:38:16.728000 | Price: 19218.65
Time: 2022-10-22 15:38:18.828000 | Price: 19218.69


Time: 2022-10-22 15:38:21.012000 | Price: 19219.05
Time: 2022-10-22 15:38:23.053000 | Price: 19218.66


CANCEL read_loop


In [39]:
trader.data

,Open,High,Low,Close,Volume,Complete
2022-10-22 15:37:00,19222.39,19223.39,19218.58,19219.56,59.16527,True
2022-10-22 15:38:00,19219.38,19219.67,19218.01,19218.66,20.10349,False
